# DS Project no. 2: Kickstarter data set  
by Sandra Schwegmann & Marvin Lorenzen

## Project Goal

Buildung the scene:  
We are a startup that helps people to find out **which probability they have to get funded** by kickstarters with their projects and what they **can do to improve their chances**.

## Data Describtion

# Data Handling

In [139]:
import pandas as pd
import glob
import numpy as np
import operator

In [140]:
# Read in data: loop over all files
path = '/Users/sandra/Documents/materials/Project-2_Kickstarter/data'
filenames = glob.glob(path + "/*.csv")
comlete_csv =[]

for i in filenames:
    comlete_csv.append(pd.read_csv(i,encoding='utf-8'))

df = pd.concat(comlete_csv, ignore_index = True)


In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 37 columns):
backers_count               209222 non-null int64
blurb                       209214 non-null object
category                    209222 non-null object
converted_pledged_amount    209222 non-null int64
country                     209222 non-null object
created_at                  209222 non-null int64
creator                     209222 non-null object
currency                    209222 non-null object
currency_symbol             209222 non-null object
currency_trailing_code      209222 non-null bool
current_currency            209222 non-null object
deadline                    209222 non-null int64
disable_communication       209222 non-null bool
friends                     300 non-null object
fx_rate                     209222 non-null float64
goal                        209222 non-null float64
id                          209222 non-null int64
is_backing                  300 

In [142]:
df.head(2)

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,315,Babalus Shoes,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",28645,US,1541459205,"{""id"":2094277840,""name"":""Lucy Conroy"",""slug"":""...",USD,$,True,...,babalus-childrens-shoes,https://www.kickstarter.com/discover/categorie...,False,False,live,1548223375,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",28645.0,international
1,47,A colorful Dia de los Muertos themed oracle de...,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",1950,US,1501684093,"{""id"":723886115,""name"":""Lisa Vollrath"",""slug"":...",USD,$,True,...,the-ofrenda-oracle-deck,https://www.kickstarter.com/discover/categorie...,True,False,successful,1504976459,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",1950.0,domestic


* data like dictonaries: 
    * category
    * creator
    * location
    * photo
    * profile
* photo will be dropped, other stay at the moment
* other object variables:
    * friends    - nothing in friends - dropped
    * is_backing - False
    * is_starred - False
    * permission - nothing in friends - dropped
* also drop source_url and urls

## Data cleaning

In [143]:
#drop first columns
df.drop(['photo','friends','permissions','currency_symbol',\
             'source_url','urls'],axis=1, inplace=True)

In [144]:
#df.location[df.location.isnull()!=False] = {}

In [145]:
# clean data from empty spaces, line breaks, etc.
dict_list1 = ['location','profile','creator']
for names in dict_list1:
    df[names] = df[names].str.replace(':null,',':None,')
    df[names] = df[names].str.replace(':false,',':False,')
    df[names] = df[names].str.replace(':true',':True')
    df[names] = df[names].str.replace('\n', '')
    df[names] = df[names].str.replace('\t', '')
    

* test which projects are still active
* test whether currency is always USD
* create a new variable project length and length per 100USD (or so)

In [146]:
print(df.shape)
len(df.state[df.state == 'live'])

(209222, 31)


7311

In [147]:

for i in df.columns:
    print(i,'', df[i][1])
    print()

backers_count  47

blurb  A colorful Dia de los Muertos themed oracle deck & guidebook for all you tarot lovers, from mixed media artist Lisa Vollrath.

category  {"id":273,"name":"Playing Cards","slug":"games/playing cards","position":4,"parent_id":12,"color":51627,"urls":{"web":{"discover":"http://www.kickstarter.com/discover/categories/games/playing%20cards"}}}

converted_pledged_amount  1950

country  US

created_at  1501684093

creator  {"id":723886115,"name":"Lisa Vollrath","slug":"lisavollrath","is_registered":None,"chosen_currency":None,"avatar":{"thumb":"https://ksr-ugc.imgix.net/assets/005/864/451/bcb1e27325c110287e4cade12433e161_original.jpg?ixlib=rb-1.1.0&w=40&h=40&fit=crop&v=1525353537&auto=format&frame=1&q=92&s=6f941b16c6c05cc5722b182874032842","small":"https://ksr-ugc.imgix.net/assets/005/864/451/bcb1e27325c110287e4cade12433e161_original.jpg?ixlib=rb-1.1.0&w=160&h=160&fit=crop&v=1525353537&auto=format&frame=1&q=92&s=eee046cfdb8810ddc961450cdf114f6d","medium":"https://ksr

In [148]:
test = df[['goal','usd_pledged','pledged']][(df.state=='successful')]
test[(df.current_currency!=df.currency)]

/Users/sandra/opt/anaconda3/envs/nf/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,goal,usd_pledged,pledged
8,375.0,529.786729,375.00
9,380.0,2516.160602,1954.00
16,4200.0,4701.023782,4230.00
17,15000.0,15328.270782,20169.00
22,1000.0,702.613237,1015.00
25,2000.0,2861.258251,2199.00
29,350.0,435.427090,376.00
32,11000.0,14580.639568,11784.00
34,2000.0,3196.518202,2055.00
35,1800.0,1325.061424,1838.32


In [149]:
# get fom creator ids -> remaining parts are ignored
# First separate creator elements
new2 = df["creator"].str.replace(", ", "_").str.split(",", n = 8, expand = True)
# now pick first column in new2 and split at :
test = new2[0].str.split(":")
# pick 2. column in test (where ids are written in)
test2 = list(map(operator.itemgetter(1), test))
# and make a new column in df with ids
df["creator_id"]=pd.Series(test2)
# convert ids to numeric values
df["creator_id"] = pd.to_numeric(df["creator_id"])

* We excluded everything after blurb as we could not manage to remove wrong entries -> has to be fixed...

In [150]:
# prepare profile data
# First separate creator elements
#new  = df["profile"].str.replace(", ", "_")
new  = df["profile"].str.replace(',"', ';;"')
new2 = new.str.split(";;", n = 16, expand = True)                                 
#new2 = new.str.replace(",000 ", "000").str.split(",", n = 16, expand = True)
# now drop column 4:
new3 = new2.drop(4, axis =1)
# change blurb to has or has no blurb
new3[5][new3[5]!='"blurb":"none"'] = '"blurb":"1"' #0 if new3[5]== "none" else 1
new3[5][new3[5]=='"blurb":"none"'] = '"blurb":"0"' #0 if new3[5]== "none" else 1

# change link_text to has or has not
#new3[10][new3[10]!='"link_text":"none"'] = '"link_text":"1"' #0 if new3[5]== "none" else 1
#new3[10][new3[10]=='"link_text":"none"'] = '"link_text":"0"' #0 if new3[5]== "none" else 1

new3[17]='"dummy":"dummy"}'
# pick columns of interest: 
#new3["profile"]= new3[[0,1,2,3,5,6,7,8,9,10,11,12,13,17]].apply(lambda x: ','.join(x), axis=1)
new3["profile"]= new3[[0,1,2,3,5,17]].apply(lambda x: ','.join(x), axis=1)
# write data in df
df["profile_test"] =new3["profile"]

In [151]:
# prepare location data
# dummy for NaN entries
location_dummy= '{"id":"","name":"","slug":"","short_name":"","displayable_name":"","localized_name":"",\
                "country":"","state":"","type":"","is_root":None,"urls":{}}'
#replace NaNs with dummy
df.location[df.location.isnull()] = location_dummy

/Users/sandra/opt/anaconda3/envs/nf/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [152]:
# extraction of data dictonaries in 'category','location'
dict_list = ['category','location','profile_test']#['category','location','profile''creator']#, null false true

for names in dict_list:
    print(names)
    new_name     = names+'new'
    suffix       = names+'_'
    df[new_name] = df[names].apply(eval)
    test         = df[new_name].apply(pd.Series)
    test         = test.add_prefix(suffix)
    df           = pd.concat([df,test],axis=1)  
 

category
location
profile_test


In [153]:
# das ging doch irgendwie schöner mit eval...
df = df[df.state!='canceled']
df = df[df.state!='live']
df = df[df.state!='suspended']
df.shape

(192664, 60)

In [154]:
# test if there are any duplicated project ids
#df.profile_test_id[df.profile_test_id.duplicated(keep=False)]==True

# these need to be tested: one succeed and one failure? why, keep it or remove it, differences...

In [155]:
pd.value_counts(df.currency)

USD    137788
GBP     21235
EUR     12399
CAD      8896
AUD      4383
MXN      2038
SEK      1281
HKD       953
NZD       848
DKK       839
SGD       624
CHF       597
NOK       453
JPY       330
Name: currency, dtype: int64

In [156]:
pd.value_counts(df.current_currency)

USD    192626
CAD        38
Name: current_currency, dtype: int64

## General Overview

In [157]:
df.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'created_at', 'creator', 'currency',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'fx_rate', 'goal', 'id', 'is_backing',
       'is_starrable', 'is_starred', 'launched_at', 'location', 'name',
       'pledged', 'profile', 'slug', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'usd_pledged', 'usd_type',
       'creator_id', 'profile_test', 'categorynew', 'category_id',
       'category_name', 'category_slug', 'category_position',
       'category_parent_id', 'category_color', 'category_urls', 'locationnew',
       'location_id', 'location_name', 'location_slug', 'location_short_name',
       'location_displayable_name', 'location_localized_name',
       'location_country', 'location_state', 'location_type',
       'location_is_root', 'location_urls', 'profile_testnew',
       'profile_test_id', 'profile_test_pro

In [158]:
df2 = df
df2.drop(['category','location','locationnew','profile','profile_test','profile_testnew','creator','profile_test_dummy'], axis=1,inplace=True)

In [159]:
df2.head()


,backers_count,blurb,converted_pledged_amount,country,created_at,currency,currency_trailing_code,current_currency,deadline,disable_communication,...,location_country,location_state,location_type,location_is_root,location_urls,profile_test_id,profile_test_project_id,profile_test_state,profile_test_state_changed_at,profile_test_blurb
1,47,A colorful Dia de los Muertos themed oracle de...,1950,US,1501684093,USD,True,USD,1504976459,False,...,US,TX,Town,False,{'web': {'discover': 'https://www.kickstarter....,3094785,3094785,active,1505402530,1
2,271,"Electra's long awaited, eclectic Debut Pop/Roc...",22404,US,1348987533,USD,True,USD,1371013395,False,...,US,CA,Suburb,False,{'web': {'discover': 'https://www.kickstarter....,359847,359847,inactive,1425915819,1
3,3,The Mist of Tribunal is a turn-based card game...,165,GB,1483780271,GBP,False,USD,1489425776,False,...,LT,Kaunas County,Town,False,{'web': {'discover': 'https://www.kickstarter....,2825329,2825329,inactive,1483780271,1
4,3,"Livng with a brain impairment, what its like t...",2820,US,1354817071,USD,True,USD,1357763527,False,...,US,MI,Town,False,{'web': {'discover': 'https://www.kickstarter....,417385,417385,inactive,1425915823,1
5,35,Annapolis Chamber Players is a non-for profit ...,3725,US,1414172150,USD,True,USD,1430533546,False,...,US,MD,Town,False,{'web': {'discover': 'https://www.kickstarter....,1465941,1465941,inactive,1425915880,1


In [160]:
df.is_backing[df.is_backing.isnull()==False] 

Series([], Name: is_backing, dtype: object)

## Missing data

In [161]:
missing = pd.DataFrame(df2.isnull().sum(),columns=['Number'])
missing[missing.Number!=0]

,Number
blurb,2
is_backing,192664
is_starred,192664
usd_type,38
category_parent_id,8276
location_slug,459
location_state,57
location_is_root,215


In [162]:
df2.columns

Index(['backers_count', 'blurb', 'converted_pledged_amount', 'country',
       'created_at', 'currency', 'currency_trailing_code', 'current_currency',
       'deadline', 'disable_communication', 'fx_rate', 'goal', 'id',
       'is_backing', 'is_starrable', 'is_starred', 'launched_at', 'name',
       'pledged', 'slug', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'usd_pledged', 'usd_type',
       'creator_id', 'categorynew', 'category_id', 'category_name',
       'category_slug', 'category_position', 'category_parent_id',
       'category_color', 'category_urls', 'location_id', 'location_name',
       'location_slug', 'location_short_name', 'location_displayable_name',
       'location_localized_name', 'location_country', 'location_state',
       'location_type', 'location_is_root', 'location_urls', 'profile_test_id',
       'profile_test_project_id', 'profile_test_state',
       'profile_test_state_changed_at', 'profile_test_blurb'],
      dtype='ob

In [163]:
df2.shape

(192664, 52)

In [164]:
abc= df2.deadline-df2.launched_at
abc.describe()

count    1.926640e+05
mean     2.798775e+06
std      1.006330e+06
min      8.640000e+04
25%      2.588400e+06
50%      2.592000e+06
75%      2.905085e+06
max      8.044324e+06
dtype: float64

In [ ]:
new2[new2[6].str.contains("she was asking for it")]
#df.profile[119063]

## Create new variables

### Goal in USD

In [167]:
df2['USD_goal'] = df2.goal*df2.static_usd_rate

### Project Duration and duration per 100 USD

### Actual duration until goal has been reached if has been reached
* see if there is a relationship between success and timing

### Has Blurb

Maybe we should not use this one... only two missing entries of which both have a fail-> results are not clear! Using instead length of Blurb?

In [168]:
df2['hasBlurb'] = 1
df2.hasBlurb[df2.blurb.isnull()] = 0
df2.hasBlurb.describe()

/Users/sandra/opt/anaconda3/envs/nf/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    192664.000000
mean          0.999990
std           0.003222
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: hasBlurb, dtype: float64

In [169]:
df2.state[df2.hasBlurb==0]

65168     failed
108662    failed
Name: state, dtype: object

In [170]:
df2['blurbLength'] = df2.blurb.str.len()

## Remove everything that is not needed anymore

remove:
* is_backing and is_starred as there are no entries left
* goal as it has been converted to USD_goal and us_static_rate as it is not needed anymore
* converted_pledge_amount and pledged as we have usd_pledge
* blurb as we have has or has not blurb
* launched_at, deadline
* blurb, as it has been replaced by length of Blurb

In [179]:
df2.drop(['is_backing','is_starred','goal','static_usd_rate','converted_pledged_amount',\
          'launched_at', 'deadline','blurb','pledged','categorynew','category_urls',\
         'location_urls','profile_test_blurb'], axis=1,inplace=True)

In [181]:
df2.columns

Index(['backers_count', 'country', 'created_at', 'currency',
       'currency_trailing_code', 'current_currency', 'disable_communication',
       'fx_rate', 'id', 'is_starrable', 'name', 'pledged', 'slug', 'spotlight',
       'staff_pick', 'state', 'state_changed_at', 'usd_pledged', 'usd_type',
       'creator_id', 'categorynew', 'category_id', 'category_name',
       'category_slug', 'category_position', 'category_parent_id',
       'category_color', 'category_urls', 'location_id', 'location_name',
       'location_slug', 'location_short_name', 'location_displayable_name',
       'location_localized_name', 'location_country', 'location_state',
       'location_type', 'location_is_root', 'location_urls', 'profile_test_id',
       'profile_test_project_id', 'profile_test_state',
       'profile_test_state_changed_at', 'profile_test_blurb', 'USD_goal',
       'hasBlurb', 'blurbLength'],
      dtype='object')